In [1]:
import pandas as pd

# ----- 경로 -----
input_path  = "/Users/minwoo/Desktop/softeer/data_engineering_course_materials/missions/final_project/data/172_시간대별_정류장_노선_추정승하차+운행횟수+순서.csv"
master_path = "/Users/minwoo/Desktop/softeer/data_engineering_course_materials/missions/final_project/data/서울시 노선 정류장마스터 정보_utf8.csv"
output_path = "/Users/minwoo/Desktop/softeer/data_engineering_course_materials/missions/final_project/data/172_시간대별_정류장_노선_추정승하차+운행횟수+순서+링크거리.csv"

# ----- 로드 -----
df = pd.read_csv(input_path, encoding="utf-8-sig")
master = pd.read_csv(master_path, encoding="utf-8-sig")

# ----- 타입 정리 (조인 키 통일) -----
df['노선ID'] = df['노선ID'].astype(str)
df['정류장_ID'] = df['정류장_ID'].astype(str)
df['정류장_순서'] = pd.to_numeric(df['정류장_순서'], errors='coerce')

master = master.rename(columns={'노선_ID':'노선ID'})
master['노선ID'] = master['노선ID'].astype(str)
master['정류장_ID'] = master['정류장_ID'].astype(str)
master['정류장_순서'] = pd.to_numeric(master['정류장_순서'], errors='coerce')

# ----- 마스터 중복 해소: 동일 키에서 링크_구간거리(m)가 있는(큰) 값 우선 -----
master_sorted = master.sort_values(['노선ID','정류장_ID','정류장_순서','링크_구간거리(m)'], ascending=[True, True, True, False])
master_dedup = master_sorted.drop_duplicates(subset=['노선ID','정류장_ID','정류장_순서'])

# ----- 조인 -----
df_merged = pd.merge(
    df,
    master_dedup[['노선ID','정류장_ID','정류장_순서','링크_구간거리(m)']],
    on=['노선ID','정류장_ID','정류장_순서'],
    how='left'
)

# (옵션) 거리 결측 기본값 지정 원하면 사용
# df_merged['링크_구간거리(m)'] = df_merged['링크_구간거리(m)'].fillna(300.0)

# ----- 저장 & 확인 -----
df_merged.to_csv(output_path, index=False, encoding="utf-8-sig")
print("✅ 저장 완료:", output_path)
print(df_merged.head(10))
print("거리 결측 수:", df_merged['링크_구간거리(m)'].isna().sum())

✅ 저장 완료: /Users/minwoo/Desktop/softeer/data_engineering_course_materials/missions/final_project/data/172_시간대별_정류장_노선_추정승하차+운행횟수+순서+링크거리.csv
        기준_날짜       노선ID     정류장_ID  시간      승차인원  하차인원  노선번호             역명  \
0  2025-06-01  100100037  110000387   0  0.000000   0.0   172  한성여객종점(00001)   
1  2025-06-01  100100037  110000387   1  0.000000   0.0   172  한성여객종점(00001)   
2  2025-06-01  100100037  110000387   2  0.000000   0.0   172  한성여객종점(00001)   
3  2025-06-01  100100037  110000387   3  1.785714   0.0   172  한성여객종점(00001)   
4  2025-06-01  100100037  110000387   4  0.583784   0.0   172  한성여객종점(00001)   
5  2025-06-01  100100037  110000387   5  0.865979   0.0   172  한성여객종점(00001)   
6  2025-06-01  100100037  110000387   6  3.647799   0.0   172  한성여객종점(00001)   
7  2025-06-01  100100037  110000387   7  8.750000   0.0   172  한성여객종점(00001)   
8  2025-06-01  100100037  110000387   8  2.964286   0.0   172  한성여객종점(00001)   
9  2025-06-01  100100037  110000387   9  1.712329   0.0   17

In [2]:
import pandas as pd

# CSV 읽기
df = pd.read_csv("./data/172_시간대별_정류장_노선_추정승하차+운행횟수+순서+링크거리.csv")

# 평균 거리 계산 (순서 1 제외)
mean_distance = df.loc[df['정류장_순서'] != 1, '링크_구간거리(m)'].mean()

# NaN 처리: NaN이면 평균값으로
df['링크_구간거리(m)'] = df['링크_구간거리(m)'].fillna(mean_distance)

# 첫 정류장은 무조건 거리 0
df.loc[df['정류장_순서'] == 1, '링크_구간거리(m)'] = 0

# 저장
df.to_csv("./data/172_시간대별_정류장_노선_추정승하차+운행횟수+순서+링크거리_보정.csv", index=False, encoding='utf-8-sig')

print(f"평균 링크 거리: {mean_distance:.2f} m")

평균 링크 거리: 467.96 m


In [1]:
import pandas as pd

PRED_CSV = "/Users/minwoo/Desktop/softeer/data_engineering_course_materials/missions/final_project/data/172_LSTM_preds_정류장별_시간별_승하차_예측.csv"
pred_df = pd.read_csv(PRED_CSV, parse_dates=['기준_날짜'])
pred_26 = pred_df[pred_df['기준_날짜'].dt.date == pd.to_datetime("2025-06-26").date()].copy()

# 키 단위 중복 카운트
dup_pred = (pred_26.groupby(['기준_날짜','시간','정류장_순서'])
                    .size().reset_index(name='rows'))
print("🔎 pred_df에서 (날짜·시간·순서)별 행 수 분포:")
print(dup_pred['rows'].value_counts().head(10))  # 대부분 1이어야 정상
print("예: rows>1인 상위 10개")
print(dup_pred[dup_pred['rows']>1].head(10))

🔎 pred_df에서 (날짜·시간·순서)별 행 수 분포:
rows
1    2496
Name: count, dtype: int64
예: rows>1인 상위 10개
Empty DataFrame
Columns: [기준_날짜, 시간, 정류장_순서, rows]
Index: []


In [2]:
if '정류장_ID' in pred_26.columns:
    dup_order = (pred_26.groupby(['기준_날짜','시간','정류장_순서'])['정류장_ID']
                        .nunique().reset_index(name='unique_stop_ids'))
    print("🔎 같은 순서에 붙은 정류장_ID 개수 분포:")
    print(dup_order['unique_stop_ids'].value_counts().head(10))
    print("예: unique_stop_ids>1 사례:")
    print(dup_order[dup_order['unique_stop_ids']>1].head(10))

🔎 같은 순서에 붙은 정류장_ID 개수 분포:
unique_stop_ids
1    2496
Name: count, dtype: int64
예: unique_stop_ids>1 사례:
Empty DataFrame
Columns: [기준_날짜, 시간, 정류장_순서, unique_stop_ids]
Index: []


In [3]:
# test_LSTM_v1.py의 simulate_passes 내부 로직을 재사용하기 어렵다면,
# 실행 후 내부에서 생성된 avg/M_df를 파일로 덤프해 확인하는 것도 방법.
# 간단 체크: now_departures 원자료가 26일만 특이한지
INPUT_CSV = "/Users/minwoo/Desktop/softeer/data_engineering_course_materials/missions/final_project/data/172_시간대별_정류장_노선_추정승하차+운행횟수+순서+링크거리_보정.csv"
raw = pd.read_csv(INPUT_CSV, parse_dates=['기준_날짜'])
raw['시간'] = raw['시간'].astype(int)
first_stop_id = (raw.sort_values('정류장_순서')
                   .groupby('정류장_ID')['정류장_순서'].min()
                   .sort_values().index[0])
dep_26 = (raw[(raw['기준_날짜'].dt.date==pd.to_datetime('2025-06-26').date()) &
              (raw['정류장_ID']==first_stop_id)]
          .groupby(['기준_날짜','시간'])['운행횟수'].sum().reset_index())
print(dep_26.head(24))

        기준_날짜  시간  운행횟수
0  2025-06-26   0   0.0
1  2025-06-26   1   0.0
2  2025-06-26   2   0.0
3  2025-06-26   3   1.0
4  2025-06-26   4   7.0
5  2025-06-26   5   7.0
6  2025-06-26   6   6.0
7  2025-06-26   7   5.0
8  2025-06-26   8   5.0
9  2025-06-26   9   5.0
10 2025-06-26  10   6.0
11 2025-06-26  11   5.0
12 2025-06-26  12   7.0
13 2025-06-26  13   6.0
14 2025-06-26  14   5.0
15 2025-06-26  15   4.0
16 2025-06-26  16   2.0
17 2025-06-26  17   4.0
18 2025-06-26  18   5.0
19 2025-06-26  19   6.0
20 2025-06-26  20   6.0
21 2025-06-26  21   5.0
22 2025-06-26  22   4.0
23 2025-06-26  23   0.0


In [4]:
from math import ceil
CAP = 33*1.5

# pred_totals와 passes merge 이후 형태를 흉내 낸 집계(간이)
pred_sum_26 = (pred_26.groupby(['기준_날짜','시간','정류장_순서'])
                      [['예측_승차인원','예측_하차인원']]
                      .sum().reset_index())

# 시간별 통과 버스수(첫 정류장 기준) 만으로는 정확도가 낮아,
# 가능하면 simulate_passes() 내 avg_best를 csv로 저장해 분석하는 게 최선.
# 여기서는 대략적인 hotspot만 본다 (통과버스수는 raw에서 추정)
M_26 = (raw[raw['기준_날짜'].dt.date==pd.to_datetime('2025-06-26').date()]
        .groupby(['기준_날짜','시간','정류장_순서'])['운행횟수'].sum()
        .reset_index(name='통과버스수'))
hot = (pred_sum_26.merge(M_26, on=['기준_날짜','시간','정류장_순서'], how='left')
                .fillna({'통과버스수':1}))
hot['버스당_탑승추정'] = hot['예측_승차인원']/hot['통과버스수']
hot['총수요'] = hot['버스당_탑승추정'] * hot['통과버스수']
hot['필요_총버스'] = hot['총수요'].apply(lambda x: ceil(x/CAP) if x>0 else 0)
hot['추가_배차'] = (hot['필요_총버스'] - hot['통과버스수']).clip(lower=0)
print(hot.sort_values('추가_배차', ascending=False).head(20))

          기준_날짜  시간  정류장_순서    예측_승차인원    예측_하차인원  통과버스수  버스당_탑승추정        총수요  \
0    2025-06-26   0       1   1.344515   0.000000    0.0       inf        NaN   
1667 2025-06-26  16       4   4.048194   1.467843    4.0  1.012048   4.048194   
1660 2025-06-26  15     101   0.834389  12.805170    6.0  0.139065   0.834389   
1661 2025-06-26  15     102   0.319340  24.363321    6.0  0.053223   0.319340   
1662 2025-06-26  15     103   0.060870  14.448785    6.0  0.010145   0.060870   
1663 2025-06-26  15     104   0.000000   3.047399    5.0  0.000000   0.000000   
1664 2025-06-26  16       1   2.577582   0.498102    2.0  1.288791   2.577582   
1665 2025-06-26  16       2  25.781504   0.522103    4.0  6.445376  25.781504   
1666 2025-06-26  16       3  23.531076   0.179740    5.0  4.706215  23.531076   
1668 2025-06-26  16       5   9.426606   4.025161    4.0  2.356652   9.426606   
1658 2025-06-26  15      99  14.378076   6.355663    5.0  2.875615  14.378076   
1669 2025-06-26  16       6 

In [1]:
import pandas as pd
import numpy as np
from math import ceil

# ========= 사용자 설정 =========
PRED_CSV   = "/Users/minwoo/Desktop/softeer/data_engineering_course_materials/missions/final_project/data/172_LSTM_preds_정류장별_시간별_승하차_예측_혼잡도+추가배차.csv"
HYBRID_CSV = "/Users/minwoo/Desktop/softeer/data_engineering_course_materials/missions/final_project/result/hybrid_tau_lookup.csv"

SEAT_CAP = 33.0               # 좌석(버스당 '여유' 임계)
CROWD_FACTOR = 1.5            # 혼잡 임계(좌석*1.5)
THRESH_OK = SEAT_CAP
THRESH_CROWD = SEAT_CAP * CROWD_FACTOR

ARRIVAL_MINUTE = 30           # 목표 도착 분(정시)
HEADWAY_GRID_MIN = 10         # 출발 분 스냅(10분 격자)

# ========= 로드 =========
pred = pd.read_csv(PRED_CSV)
hyb  = pd.read_csv(HYBRID_CSV)

# 타입/컬럼 표준화
pred["기준_날짜"] = pd.to_datetime(pred["기준_날짜"]).dt.date
pred["시간"] = pred["시간"].astype(int)
pred["정류장_순서"] = pred["정류장_순서"].astype(int)
if "버스당_onboard_추정" not in pred.columns:
    raise ValueError("PRED_CSV에 '버스당_onboard_추정' 컬럼이 필요합니다.")
if "통과버스수" not in pred.columns:
    raise ValueError("PRED_CSV에 '통과버스수' 컬럼이 필요합니다.")
if "역명" not in pred.columns:
    pred["역명"] = ""

# 하이브리드 τ 인덱스 키 통일
hyb = hyb.rename(columns={"도착_시간":"시간"})
hyb["정류장_순서"] = hyb["정류장_순서"].astype(int)
hyb["시간"] = hyb["시간"].astype(int)

def snap_minute(m: int, grid: int = HEADWAY_GRID_MIN) -> int:
    return m if grid <= 1 else (m // grid) * grid

def plan_departure(date_str: str, hour: int, stop_seq: int, weekday_group: str) -> dict:
    """ 혼잡(목표) [date, hour, stop_seq] 도착을 ARRIVAL_MINUTE에 맞추려면 몇 시 출발해야 하는지 """
    key = (stop_seq, hour, weekday_group)
    row = hyb[(hyb["정류장_순서"]==stop_seq) & (hyb["시간"]==hour) & (hyb["요일그룹"]==weekday_group)]
    if row.empty:
        return {"ok": False, "reason": f"hybrid_tau에 {key} 없음"}
    tau = float(row["tau_final_min"].iloc[0])
    src = str(row["tau_source"].iloc[0])

    arrival = pd.Timestamp(f"{date_str} {hour:02d}:{ARRIVAL_MINUTE:02d}:00")
    depart  = arrival - pd.Timedelta(minutes=tau)
    depart_snapped = depart.replace(minute=snap_minute(depart.minute), second=0, microsecond=0)

    return {
        "ok": True,
        "tau_min": tau,
        "tau_source": src,
        "depart_time": depart,
        "depart_time_snapped": depart_snapped,
        "depart_hour": int(depart_snapped.hour)
    }

def weekday_group_of_date(d: pd.Timestamp) -> str:
    return "평일" if pd.to_datetime(d).weekday() < 5 else "주말"

def summarize_hours_for_day(day_df: pd.DataFrame):
    """ 하루에 대해 시간별 지표 요약: 최댓 온보드, 피크 정류장, 버스수 등 """
    grp = day_df.groupby(["시간","정류장_순서"], as_index=False).agg(
        max_onboard=("버스당_onboard_추정", "max"),
        bus_count=("통과버스수", "max"),
        stop_name=("역명", "max")
    )
    # 시간별 최악 정류장(온보드 최대)
    idx = grp.groupby("시간")["max_onboard"].idxmax()
    worst = grp.loc[idx].reset_index(drop=True)

    # 서비스 시작/종료시간
    service = day_df.groupby("시간")["통과버스수"].max()
    active_hours = sorted([int(h) for h,v in service.items() if v>0])
    if len(active_hours)==0:
        first_hr, last_hr = None, None
    else:
        first_hr, last_hr = min(active_hours), max(active_hours)

    return worst, first_hr, last_hr

def donor_hours_for_day(day_df: pd.DataFrame, first_hr: int, last_hr: int):
    """ 도너 후보: 여유(≤33)이며 1대 빼도 여유 유지 & 운영시간 ±3h 제약 """
    grp = day_df.groupby(["시간","정류장_순서"], as_index=False).agg(
        max_onboard=("버스당_onboard_추정", "max"),
        bus_count=("통과버스수", "max")
    )
    # 시간별 최고 온보드, 해당 시간 버스수
    worst = grp.groupby("시간").agg(
        max_onboard=("max_onboard","max"),
        bus_count=("bus_count","max")
    ).reset_index()

    if first_hr is None:
        return worst.iloc[0:0].copy()

    allowed = worst[
        (worst["시간"] >= first_hr + 3) &
        (worst["시간"] <= last_hr  - 3)
    ].copy()

    # 1대 빼도 여유 유지?  new_onboard ≈ old_onboard * N/(N-1)
    allowed["can_remove"] = False
    mask = allowed["bus_count"] >= 2
    allowed.loc[mask, "can_remove"] = (allowed.loc[mask, "max_onboard"] * 
                                       (allowed.loc[mask, "bus_count"] / (allowed.loc[mask, "bus_count"] - 1))
                                      ) <= THRESH_OK

    donors = allowed[(allowed["max_onboard"] <= THRESH_OK) & (allowed["can_remove"])].copy()
    # 여유가 큰 순(= 제거 안전한 순)으로 정렬
    donors["post_remove_max"] = donors["max_onboard"] * (donors["bus_count"]/(donors["bus_count"]-1))
    donors = donors.sort_values(["post_remove_max","시간"]).reset_index(drop=True)
    donors["available_to_move"] = 1  # 시간당 1대씩만 이동(보수적)
    return donors[["시간","bus_count","max_onboard","available_to_move"]]

def needed_additional_buses(max_onboard: float, current_buses: float) -> int:
    """
    목표: 버스당 온보드 <= THRESH_CROWD(보통 이하)로 낮추기
    현재 per-bus b, 버스 N → 추가 d 후 ≈ b * N/(N+d) <= THRESH_CROWD
    -> d >= ceil(N*(b/THRESH_CROWD - 1)), 최소 0
    """
    if current_buses <= 0:
        return 0
    d = ceil(max(0.0, current_buses*(max_onboard/THRESH_CROWD - 1.0)))
    return int(d)

def reallocate_for_day(day: pd.Timestamp, day_df: pd.DataFrame) -> pd.DataFrame:
    """
    하루 단위 재배치:
      1) 혼잡 시간대(최대 온보드 > 혼잡 임계)별로 부족 대수 산출
      2) 도너 시간대에서 1대씩 꺼내 target으로 이동
      3) 하이브리드 τ로 출발시각 역산(목표 도착: 해당 시간, 최악 정류장, :ARRIVAL_MINUTE)
    """
    worst_by_hour, first_hr, last_hr = summarize_hours_for_day(day_df)
    if first_hr is None:
        return pd.DataFrame(columns=[
            "기준_날짜","from_hour","to_hour","pivot_stop_seq","pivot_stop_name",
            "needed_before","moved","needed_after","depart_time_snapped","tau_min","tau_source"
        ])

    # 혼잡 타깃(심한 순)
    worst_by_hour["shortage"] = worst_by_hour.apply(
        lambda r: needed_additional_buses(r["max_onboard"], r["bus_count"]),
        axis=1
    )
    targets = worst_by_hour[worst_by_hour["shortage"] > 0].copy()
    if targets.empty:
        return pd.DataFrame(columns=[
            "기준_날짜","from_hour","to_hour","pivot_stop_seq","pivot_stop_name",
            "needed_before","moved","needed_after","depart_time_snapped","tau_min","tau_source"
        ])

    # 도너 목록
    donors = donor_hours_for_day(day_df, first_hr, last_hr)
    if donors.empty:
        # 도너 없음
        out = []
        for _,t in targets.sort_values("shortage",ascending=False).iterrows():
            out.append({
                "기준_날짜": str(day.date()),
                "from_hour": None,
                "to_hour": None,
                "pivot_stop_seq": int(worst_by_hour.loc[worst_by_hour["시간"]==t["시간"],"정류장_순서"].values[0]),
                "pivot_stop_name": str(day_df[day_df["시간"]==t["시간"]].sort_values("버스당_onboard_추정",ascending=False)["역명"].iloc[0]),
                "needed_before": int(t["shortage"]),
                "moved": 0,
                "needed_after": int(t["shortage"]),
                "depart_time_snapped": None,
                "tau_min": None,
                "tau_source": None,
            })
        return pd.DataFrame(out)

    # 요일 그룹
    yoil = weekday_group_of_date(pd.to_datetime(day))

    # 타깃별로 그리디 재배치
    plan_rows = []
    donors = donors.copy()
    for _, t in targets.sort_values("shortage", ascending=False).iterrows():
        tgt_hour = int(t["시간"])
        pivot_row = day_df[day_df["시간"]==tgt_hour].sort_values("버스당_onboard_추정", ascending=False).iloc[0]
        pivot_seq = int(pivot_row["정류장_순서"])
        pivot_name = str(pivot_row["역명"])
        need = int(t["shortage"])

        moved_here = 0
        # 도너를 안전도 높은 순으로 순회
        for di in donors.index.tolist():
            if need <= 0:
                break
            if donors.loc[di,"available_to_move"] <= 0:
                continue

            from_hour = int(donors.loc[di,"시간"])
            # 출발시각 역산
            plan = plan_departure(str(day.date()), tgt_hour, pivot_seq, yoil)
            if not plan["ok"]:
                continue
            to_hour = int(plan["depart_hour"])

            # 1대 이동 실행(보수적)
            donors.loc[di,"available_to_move"] -= 1
            moved_here += 1

            plan_rows.append({
                "기준_날짜": str(day.date()),
                "from_hour": from_hour,
                "to_hour": to_hour,
                "pivot_stop_seq": pivot_seq,
                "pivot_stop_name": pivot_name,
                "needed_before": need,
                "moved": 1,
                "needed_after": max(0, need-1),
                "depart_time_snapped": str(plan["depart_time_snapped"]),
                "tau_min": round(plan["tau_min"],3),
                "tau_source": plan["tau_source"],
            })

            need -= 1

    if not plan_rows:
        return pd.DataFrame(columns=[
            "기준_날짜","from_hour","to_hour","pivot_stop_seq","pivot_stop_name",
            "needed_before","moved","needed_after","depart_time_snapped","tau_min","tau_source"
        ])
    plan_df = pd.DataFrame(plan_rows)

    # 같은 타깃 시간대에 대해 여러 이동이 있었으면 마지막 상태 기준으로 묶어서 볼 수도 있음
    return plan_df

# ========= 실행: 모든 날짜에 대해 재배치 제안 =========
plans = []
for day, day_df in pred.groupby("기준_날짜"):
    # 한 날짜 안에서 정류장/시간 별로 예측이 여러 행이면 집계 안전용(최댓 온보드, 최대 버스수)
    # (이미 그 형태라면 영향 없음)
    plans.append(reallocate_for_day(pd.Timestamp(day), day_df))

plans = pd.concat([p for p in plans if not p.empty], ignore_index=True) if len(plans)>0 else pd.DataFrame()

if plans.empty:
    print("✅ 재배치가 필요한 혼잡 시간대가 발견되지 않았습니다. (또는 도너 없음)")
else:
    # 보기 좋게 정렬/표시
    plans = plans.sort_values(["기준_날짜","to_hour","from_hour","pivot_stop_seq"]).reset_index(drop=True)
    print("🧭 재배치 제안(상위 30행):")
    display(plans.head(30))

    # 일자별 요약
    summary = plans.groupby("기준_날짜").agg(
        moved_total=("moved","sum"),
        unique_to_hours=("to_hour","nunique"),
        unique_from_hours=("from_hour","nunique")
    ).reset_index()
    print("\n📊 일자별 이동 요약:")
    display(summary)

🧭 재배치 제안(상위 30행):


,기준_날짜,from_hour,to_hour,pivot_stop_seq,pivot_stop_name,needed_before,moved,needed_after,depart_time_snapped,tau_min,tau_source
0,2025-06-28,5,14,85,길음2동주민센터(00085),2,1,1,2025-06-28 14:30:00.000000001,114.109,observed_p60
1,2025-06-28,14,14,85,길음2동주민센터(00085),1,1,0,2025-06-28 14:30:00.000000001,114.109,observed_p60



📊 일자별 이동 요약:


,기준_날짜,moved_total,unique_to_hours,unique_from_hours
0,2025-06-28,2,1,2


In [5]:
import numpy as np
import pandas as pd
from math import ceil

# -----------------------------
# 설정값 (좌석수/임계치/헤드웨이)
# -----------------------------
SEAT = 33
TH_YEOYU = SEAT * 1.0      # 여유 임계치 (<= 33)
TH_BOTONG = SEAT * 1.5      # 보통 임계치 (<= 49.5면 '보통'으로 간주)
# headway, 보호시간 등은 호출부에서 넘겨주세요.

# -----------------------------
# 요일 그룹핑: 0=월 ~ 6=일
# -----------------------------
def get_dow_group(dow: int) -> str:
    return "주말" if dow >= 5 else "주중"

# -----------------------------
# pred_df -> 혼잡 타깃(needed_before)
# pred_df 스키마:
#  ['기준_날짜','시간','정류장_ID','정류장_순서','역명',
#   '통과버스수','예측_승차_총','예측_하차_총','버스당_승차','버스당_하차','버스당_onboard_추정']
# -----------------------------
def build_hourly_need_from_pred(pred_df: pd.DataFrame) -> pd.DataFrame:
    df = pred_df.copy()

    # 타입 정리
    df["기준_날짜"] = pd.to_datetime(df["기준_날짜"])
    df["시간"] = df["시간"].astype(int)
    df["통과버스수"] = df["통과버스수"].fillna(0).astype(float)

    # 버스 없는 시간대 제외(필요 0으로 떨어질 것이므로 계산 낭비 방지)
    df = df[df["통과버스수"] > 0].copy()

    # 시단별 피벗 정류장(버스당_onboard_추정이 최대)
    idx = df.groupby(["기준_날짜","시간"])["버스당_onboard_추정"].idxmax()
    pivot = df.loc[idx, [
        "기준_날짜","시간","정류장_순서","역명","통과버스수","버스당_onboard_추정"
    ]].rename(columns={
        "정류장_순서":"pivot_stop_seq",
        "역명":"pivot_stop_name",
        "통과버스수":"curr_buses",
        "버스당_onboard_추정":"bus_onboard"
    })

    # 총수요(보수적으로: 최악 정류장 기준)
    pivot["total_demand"] = pivot["bus_onboard"] * pivot["curr_buses"]

    # 혼잡 해소 위해 필요한 추가대수 (보통 임계치까지 낮추는 목표)
    # ceil(total / TH_BOTONG) - curr_buses
    pivot["needed_before"] = np.maximum(
        0,
        np.ceil(pivot["total_demand"] / TH_BOTONG).astype(int) - pivot["curr_buses"].astype(int)
    )

    # 출력 형식
    hourly_need = pivot[[
        "기준_날짜","시간","pivot_stop_seq","pivot_stop_name",
        "curr_buses","total_demand","bus_onboard","needed_before"
    ]].rename(columns={"시간":"시간"})  # 그대로 둠 (나중에 to_hour로 rename 예정)

    return hourly_need

# -----------------------------
# pred_df -> 여유 도너(spare)
# -----------------------------
def build_hourly_supply_from_pred(pred_df: pd.DataFrame) -> pd.DataFrame:
    df = pred_df.copy()

    # 타입 정리
    df["기준_날짜"] = pd.to_datetime(df["기준_날짜"])
    df["시간"] = df["시간"].astype(int)
    df["통과버스수"] = df["통과버스수"].fillna(0).astype(float)

    df = df[df["통과버스수"] > 0].copy()

    # 시단별 피벗 정류장(버스당_onboard_추정이 최대)
    idx = df.groupby(["기준_날짜","시간"])["버스당_onboard_추정"].idxmax()
    pivot = df.loc[idx, [
        "기준_날짜","시간","정류장_순서","역명","통과버스수","버스당_onboard_추정"
    ]].rename(columns={
        "정류장_순서":"pivot_stop_seq",
        "역명":"pivot_stop_name",
        "통과버스수":"curr_buses",
        "버스당_onboard_추정":"bus_onboard"
    })

    # 총수요(보수적으로: 최악 정류장 기준)
    pivot["total_demand"] = pivot["bus_onboard"] * pivot["curr_buses"]

    # '여유' 유지하며 뺄 수 있는 최대 대수:
    # curr_buses - ceil(total_demand / TH_YEOYU)
    pivot["spare"] = np.maximum(
        0,
        pivot["curr_buses"].astype(int) - np.ceil(pivot["total_demand"] / TH_YEOYU).astype(int)
    ).astype(int)

    hourly_supply = pivot[[
        "기준_날짜","시간","pivot_stop_seq","pivot_stop_name",
        "curr_buses","total_demand","bus_onboard","spare"
    ]].rename(columns={"시간":"시간"})

    return hourly_supply

# ----------------------------------------------------
# 보호구간(첫차+3h, 막차-3h) 계산: 기점(정류장_순서==1) 기준
# schedule: 최소 ['기준_날짜','도착_시간','정류장_순서']
# ----------------------------------------------------
def compute_protected_windows(schedule: pd.DataFrame, protect_h=3) -> pd.DataFrame:
    sc = schedule.copy()
    sc["기준_날짜"] = pd.to_datetime(sc["기준_날짜"])
    sc["도착_시간"] = sc["도착_시간"].astype(int)

    first_last = (
        sc[sc["정류장_순서"]==1]
        .groupby("기준_날짜")["도착_시간"]
        .agg(first_hour="min", last_hour="max")
        .reset_index()
    )
    first_last["protect_start_hi"] = first_last["first_hour"] + protect_h
    first_last["protect_end_lo"]   = first_last["last_hour"]  - protect_h
    return first_last

# ----------------------------------------------------
# 혼잡 타깃/여유 공여 시간대 만들기
# hourly_need: [기준_날짜, 시간, pivot_stop_seq, pivot_stop_name, needed_before, ...]
# hourly_supply: [기준_날짜, 시간, spare, ...]
# ----------------------------------------------------
def build_targets_and_donors(hourly_need: pd.DataFrame,
                             hourly_supply: pd.DataFrame,
                             schedule: pd.DataFrame,
                             protect_h=3,
                             debug=True):
    fl = compute_protected_windows(schedule, protect_h=protect_h)

    # 타깃: 혼잡 시단 (needed_before>0) + 보호구간 바깥
    tgt = (hourly_need.copy()
           .rename(columns={"시간":"to_hour"}))
    tgt = tgt.merge(fl, on="기준_날짜", how="left")
    tgt = tgt[(tgt["needed_before"]>0) &
              (tgt["to_hour"]>=tgt["protect_start_hi"]) &
              (tgt["to_hour"]<=tgt["protect_end_lo"])].copy()

    # 도너: 여유 시단 (spare>0) + 보호구간 바깥
    don = (hourly_supply.copy()
           .rename(columns={"시간":"from_hour"}))
    don = don.merge(fl, on="기준_날짜", how="left")
    don = don[(don["spare"]>0) &
              (don["from_hour"]>=don["protect_start_hi"]) &
              (don["from_hour"]<=don["protect_end_lo"])].copy()

    if debug:
        print(f"[DEBUG] 타깃(혼잡) 시단 수: {len(tgt)} / 도너(여유) 시단 수: {len(don)}")
        print(f"      보호구간(first+{protect_h}, last-{protect_h}) 적용 후 남은 시단만 대상으로 합니다.")
        if len(tgt)==0:
            print(" - 혼잡 타깃이 모두 보호구간에 있거나 이미 해소되어 제안이 없습니다.")
        if len(don)==0:
            print(" - 여유 도너가 보호구간에 있거나 spare=0이라 제안이 제한됩니다.")

    return tgt, don

# ----------------------------------------------------
# τ lookup 이용해서 출발시각 산정 + 그리디 매칭
# hybrid_tau: [정류장_순서, 도착_시간, 요일그룹, tau_final_min, tau_source]
# ----------------------------------------------------
def plan_reallocation(tgt: pd.DataFrame,
                      don: pd.DataFrame,
                      hybrid_tau: pd.DataFrame,
                      get_dow_group,
                      headway_min=10,
                      max_moves_per_target=99,
                      debug=True) -> pd.DataFrame:
    plans = []

    # 정렬 (필요 큰 타깃 먼저, 여유 큰 도너 먼저)
    tgt = tgt.sort_values(["기준_날짜","needed_before"], ascending=[True, False]).reset_index(drop=True)
    don = don.sort_values(["기준_날짜","spare"],         ascending=[True, False]).reset_index(drop=True)

    # 타입 보정
    hybrid_tau = hybrid_tau.copy()
    # 요일그룹/정류장_순서/도착_시간 타입 일치
    if "도착_시간" in hybrid_tau.columns:
        hybrid_tau["도착_시간"] = hybrid_tau["도착_시간"].astype(int)
    if "정류장_순서" in hybrid_tau.columns:
        hybrid_tau["정류장_순서"] = hybrid_tau["정류장_순서"].astype(int)

    for day, tgt_day in tgt.groupby("기준_날짜"):
        don_day = don[don["기준_날짜"]==day].copy()
        if don_day.empty:
            if debug: print(f"[DEBUG] {day.date()} 도너 없음 → 스킵")
            continue

        dow_group = get_dow_group(pd.to_datetime(day).weekday())

        for _, row in tgt_day.iterrows():
            need = int(row["needed_before"])
            if need<=0:
                continue

            to_hour   = int(row["to_hour"])
            pivot_seq = int(row["pivot_stop_seq"])
            pivot_nm  = row.get("pivot_stop_name","")

            tau_row = hybrid_tau[
                (hybrid_tau["정류장_순서"]==pivot_seq) &
                (hybrid_tau["도착_시간"]==to_hour) &
                (hybrid_tau["요일그룹"]==dow_group)
            ]
            if tau_row.empty:
                if debug: print(f"[DEBUG] τ없음 → {day.date()} {to_hour}h seq{pivot_seq} 스킵")
                continue

            tau_min    = float(tau_row.iloc[0]["tau_final_min"])
            tau_source =       tau_row.iloc[0]["tau_source"]

            moved_total = 0
            # 도너 중 spare 큰 순으로 가져오기
            for j, d in don_day.sort_values("spare", ascending=False).iterrows():
                if moved_total >= min(need, max_moves_per_target):
                    break
                if d["spare"]<=0:
                    continue
                take = int(min(d["spare"], need - moved_total))
                if take<=0:
                    continue

                from_hour = int(d["from_hour"])
                # 피벗 정류장 목표 도착시각
                pivot_time = pd.to_datetime(str(day.date())) + pd.Timedelta(hours=to_hour)
                # 출발 = 도착 - τ
                depart_time = pivot_time - pd.Timedelta(minutes=tau_min)
                # 헤드웨이 스냅
                snap_min = (depart_time.minute // headway_min) * headway_min
                depart_time_snapped = depart_time.replace(minute=0, second=0, microsecond=0) + pd.Timedelta(minutes=snap_min)

                plans.append({
                    "기준_날짜": day.date(),
                    "from_hour": from_hour,
                    "to_hour": to_hour,
                    "pivot_stop_seq": pivot_seq,
                    "pivot_stop_name": pivot_nm,
                    "needed_before": int(need),
                    "moved": int(take),
                    "needed_after": int(need - (moved_total + take)),
                    "depart_time_snapped": depart_time_snapped,
                    "tau_min": round(tau_min,3),
                    "tau_source": tau_source
                })

                # 도너 차감
                don_day.loc[j, "spare"] -= take
                moved_total += take

            if debug and moved_total==0:
                print(f"[DEBUG] {day.date()} {to_hour}h seq{pivot_seq}: 도너 부족/제약으로 이동 0대")

    if not plans:
        return pd.DataFrame(columns=[
            "기준_날짜","from_hour","to_hour","pivot_stop_seq","pivot_stop_name",
            "needed_before","moved","needed_after","depart_time_snapped","tau_min","tau_source"
        ])

    plans = pd.DataFrame(plans).sort_values(
        ["기준_날짜","to_hour","from_hour","pivot_stop_seq"]
    ).reset_index(drop=True)
    return plans

In [7]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
from math import ceil

# ---------------------------------------
# 0) 설정값 (필요시만 수정)
# ---------------------------------------
BUS_DAY_PATH    = "/Users/minwoo/Desktop/softeer/data_engineering_course_materials/missions/final_project/result/172_버스단위_스케줄+수요분배_2025-06-01_to_2025-06-30.csv"
HYBRID_TAU_PATH = "/Users/minwoo/Desktop/softeer/data_engineering_course_materials/missions/final_project/result/hybrid_tau_lookup.csv"
OUT_PLAN_PATH   = "/Users/minwoo/Desktop/softeer/data_engineering_course_materials/missions/final_project/result/재배치_제안.csv"

# 임계치/좌석수/보호구간/헤드웨이
SEAT              = 33                     # 버스 좌석수
TH_YEOYU          = SEAT * 1.0             # 여유 임계(버스당 ≤ 33)
TH_BOTONG         = SEAT * 1.5             # 보통 임계(버스당 ≤ 49.5)
PROTECT_H         = 3                      # 첫차+3h / 막차-3h 보호
HEADWAY_GRID_MIN  = 10                     # 출발 스냅 그리드(분)

# ---------------------------------------
# 1) 요일 그룹핑
# ---------------------------------------
def get_dow_group(dow: int) -> str:
    # 0=월 ... 6=일
    return "주말" if dow >= 5 else "주중"

# ---------------------------------------
# 2) 혼잡 타깃/여유 도너 생성용 함수
#    pred_df 스키마 기대:
#  ['기준_날짜','시간','정류장_순서','역명',
#   '통과버스수','예측_승차_총','예측_하차_총','버스당_onboard_추정']
# ---------------------------------------
def build_hourly_need_from_pred(pred_df: pd.DataFrame) -> pd.DataFrame:
    df = pred_df.copy()
    df["기준_날짜"]   = pd.to_datetime(df["기준_날짜"])
    df["시간"]       = df["시간"].astype(int)
    df["통과버스수"]   = df["통과버스수"].fillna(0).astype(float)
    df = df[df["통과버스수"] > 0].copy()

    # 시단별 최악(버스당_onboard 최대) 정류장 기준으로 수요를 보수적으로 잡음
    idx = df.groupby(["기준_날짜","시간"])["버스당_onboard_추정"].idxmax()
    pivot = df.loc[idx, ["기준_날짜","시간","정류장_순서","역명","통과버스수","버스당_onboard_추정"]].rename(
        columns={
            "정류장_순서": "pivot_stop_seq",
            "역명"      : "pivot_stop_name",
            "통과버스수" : "curr_buses",
            "버스당_onboard_추정": "bus_onboard"
        }
    )
    pivot["total_demand"]  = pivot["bus_onboard"] * pivot["curr_buses"]
    pivot["needed_before"] = np.maximum(
        0,
        np.ceil(pivot["total_demand"] / TH_BOTONG).astype(int) - pivot["curr_buses"].astype(int)
    )

    hourly_need = pivot[[
        "기준_날짜","시간","pivot_stop_seq","pivot_stop_name",
        "curr_buses","total_demand","bus_onboard","needed_before"
    ]]
    return hourly_need

def build_hourly_supply_from_pred(pred_df: pd.DataFrame) -> pd.DataFrame:
    df = pred_df.copy()
    df["기준_날짜"]   = pd.to_datetime(df["기준_날짜"])
    df["시간"]       = df["시간"].astype(int)
    df["통과버스수"]   = df["통과버스수"].fillna(0).astype(float)
    df = df[df["통과버스수"] > 0].copy()

    idx = df.groupby(["기준_날짜","시간"])["버스당_onboard_추정"].idxmax()
    pivot = df.loc[idx, ["기준_날짜","시간","정류장_순서","역명","통과버스수","버스당_onboard_추정"]].rename(
        columns={
            "정류장_순서": "pivot_stop_seq",
            "역명"      : "pivot_stop_name",
            "통과버스수" : "curr_buses",
            "버스당_onboard_추정": "bus_onboard"
        }
    )
    pivot["total_demand"] = pivot["bus_onboard"] * pivot["curr_buses"]
    pivot["spare"] = np.maximum(
        0,
        pivot["curr_buses"].astype(int) - np.ceil(pivot["total_demand"] / TH_YEOYU).astype(int)
    ).astype(int)

    hourly_supply = pivot[[
        "기준_날짜","시간","pivot_stop_seq","pivot_stop_name",
        "curr_buses","total_demand","bus_onboard","spare"
    ]]
    return hourly_supply

# ---------------------------------------
# 3) 보호구간(첫차+3h/막차-3h) 계산
#    schedule: ['기준_날짜','도착_시간','정류장_순서'] 필요
# ---------------------------------------
def compute_protected_windows(schedule: pd.DataFrame, protect_h=3) -> pd.DataFrame:
    sc = schedule.copy()
    sc["기준_날짜"] = pd.to_datetime(sc["기준_날짜"])
    sc["도착_시간"] = sc["도착_시간"].astype(int)
    first_last = (
        sc[sc["정류장_순서"]==1]
        .groupby("기준_날짜")["도착_시간"]
        .agg(first_hour="min", last_hour="max")
        .reset_index()
    )
    first_last["protect_start_hi"] = first_last["first_hour"] + protect_h
    first_last["protect_end_lo"]   = first_last["last_hour"]  - protect_h
    return first_last

# ---------------------------------------
# 4) 타깃/도너 구성 (보호구간 제외)
# ---------------------------------------
def build_targets_and_donors(hourly_need: pd.DataFrame,
                             hourly_supply: pd.DataFrame,
                             schedule: pd.DataFrame,
                             protect_h=3,
                             debug=True):
    fl = compute_protected_windows(schedule, protect_h=protect_h)

    tgt = hourly_need.copy().rename(columns={"시간":"to_hour"})
    tgt = tgt.merge(fl, on="기준_날짜", how="left")
    tgt = tgt[(tgt["needed_before"]>0) &
              (tgt["to_hour"]>=tgt["protect_start_hi"]) &
              (tgt["to_hour"]<=tgt["protect_end_lo"])].copy()

    don = hourly_supply.copy().rename(columns={"시간":"from_hour"})
    don = don.merge(fl, on="기준_날짜", how="left")
    don = don[(don["spare"]>0) &
              (don["from_hour"]>=don["protect_start_hi"]) &
              (don["from_hour"]<=don["protect_end_lo"])].copy()

    if debug:
        print(f"[DEBUG] 타깃(혼잡) 시단 수: {len(tgt)} / 도너(여유) 시단 수: {len(don)}")
        print(f"      보호구간(first+{protect_h}, last-{protect_h}) 적용 후 남은 시단만 대상으로 합니다.")
        if len(tgt)==0: print(" - 혼잡 타깃 없음(보호구간/무혼잡).")
        if len(don)==0: print(" - 여유 도너 없음(보호구간/spare=0).")

    return tgt, don

# ---------------------------------------
# 5) τ 기반 출발시각 역산 + 그리디 매칭
#    hybrid_tau: ['정류장_순서','도착_시간','요일그룹','tau_final_min','tau_source']
# ---------------------------------------
def plan_reallocation(tgt: pd.DataFrame,
                      don: pd.DataFrame,
                      hybrid_tau: pd.DataFrame,
                      get_dow_group,
                      headway_min=10,
                      max_moves_per_target=99,
                      debug=True) -> pd.DataFrame:
    plans = []
    tgt = tgt.sort_values(["기준_날짜","needed_before"], ascending=[True, False]).reset_index(drop=True)
    don = don.sort_values(["기준_날짜","spare"],         ascending=[True, False]).reset_index(drop=True)

    tau = hybrid_tau.copy()
    if "도착_시간" in tau.columns:  tau["도착_시간"]   = tau["도착_시간"].astype(int)
    if "정류장_순서" in tau.columns: tau["정류장_순서"] = tau["정류장_순서"].astype(int)

    for day, tgt_day in tgt.groupby("기준_날짜"):
        don_day = don[don["기준_날짜"]==day].copy()
        if don_day.empty:
            if debug: print(f"[DEBUG] {day.date()} 도너 없음 → 스킵")
            continue

        dow_group = get_dow_group(pd.to_datetime(day).weekday())

        for _, row in tgt_day.iterrows():
            need = int(row["needed_before"])
            if need<=0: continue

            to_hour   = int(row["to_hour"])
            pivot_seq = int(row["pivot_stop_seq"])
            pivot_nm  = row.get("pivot_stop_name","")

            tau_row = tau[(tau["정류장_순서"]==pivot_seq) &
                          (tau["도착_시간"]==to_hour) &
                          (tau["요일그룹"]==dow_group)]
            if tau_row.empty:
                if debug: print(f"[DEBUG] τ없음 → {day.date()} {to_hour}h seq{pivot_seq} 스킵")
                continue

            tau_min    = float(tau_row.iloc[0]["tau_final_min"])
            tau_source =       tau_row.iloc[0]["tau_source"]

            moved_total = 0
            for j, d in don_day.sort_values("spare", ascending=False).iterrows():
                if moved_total>=min(need, max_moves_per_target): break
                if d["spare"]<=0: continue
                take = int(min(d["spare"], need - moved_total))
                if take<=0: continue

                from_hour  = int(d["from_hour"])
                pivot_time = pd.to_datetime(str(day.date())) + pd.Timedelta(hours=to_hour)
                depart_time = pivot_time - pd.Timedelta(minutes=tau_min)
                snap_min = (depart_time.minute // headway_min) * headway_min
                depart_time_snapped = depart_time.replace(minute=0, second=0, microsecond=0) + pd.Timedelta(minutes=snap_min)

                plans.append({
                    "기준_날짜": day.date(),
                    "from_hour": from_hour,
                    "to_hour": to_hour,
                    "pivot_stop_seq": pivot_seq,
                    "pivot_stop_name": pivot_nm,
                    "needed_before": int(need),
                    "moved": int(take),
                    "needed_after": int(need - (moved_total + take)),
                    "depart_time_snapped": depart_time_snapped,
                    "tau_min": round(tau_min,3),
                    "tau_source": tau_source
                })
                don_day.loc[j, "spare"] -= take
                moved_total += take

            if debug and moved_total==0:
                print(f"[DEBUG] {day.date()} {to_hour}h seq{pivot_seq}: 도너 부족/제약으로 이동 0대")

    if not plans:
        return pd.DataFrame(columns=[
            "기준_날짜","from_hour","to_hour","pivot_stop_seq","pivot_stop_name",
            "needed_before","moved","needed_after","depart_time_snapped","tau_min","tau_source"
        ])

    plans = pd.DataFrame(plans).sort_values(
        ["기준_날짜","to_hour","from_hour","pivot_stop_seq"]
    ).reset_index(drop=True)
    return plans


# ===================================================
# 메인: 입력 로드 → pred_df/schedule 구성 → 타깃/도너/플랜 산출
# ===================================================
print("📥 Load schedule+allocation…")
df = pd.read_csv(BUS_DAY_PATH)

# 도착_시간 보정
if "도착_시간" not in df.columns and "도착시각" in df.columns:
    dt = pd.to_datetime(df["도착시각"])
    df["도착_시간"] = dt.dt.hour

# schedule(보호구간 계산용)
schedule = (
    df[["기준_날짜","도착_시간","정류장_순서"]]
    .drop_duplicates()
    .reset_index(drop=True)
)

# pred_df(혼잡/여유 판단용) 만들기
group_cols = ["기준_날짜","도착_시간","정류장_순서"]
name_col = "역명_x" if "역명_x" in df.columns else ("역명" if "역명" in df.columns else None)
group_cols_named = group_cols + ([name_col] if name_col else [])

pred_df = (
    df.groupby(group_cols_named, as_index=False)
      .agg(
          통과버스수=("bus_id","nunique"),
          예측_승차_총=("총승차","sum"),
          예측_하차_총=("총하차","sum"),
          버스당_onboard_추정=("onboard_after","mean")  # 필요시 "median"
      )
).rename(columns={"도착_시간":"시간"})
if name_col:
    pred_df = pred_df.rename(columns={name_col:"역명"})

# 타입 표준화
pred_df["기준_날짜"] = pd.to_datetime(pred_df["기준_날짜"])

print("📥 Load hybrid τ…")
hybrid_tau = pd.read_csv(HYBRID_TAU_PATH)

# 혼잡 타깃/여유 도너 산출
hourly_need   = build_hourly_need_from_pred(pred_df)
hourly_supply = build_hourly_supply_from_pred(pred_df)

tgt, don = build_targets_and_donors(
    hourly_need, hourly_supply, schedule,
    protect_h=PROTECT_H,
    debug=True
)

plans = plan_reallocation(
    tgt, don, hybrid_tau,
    get_dow_group=get_dow_group,
    headway_min=HEADWAY_GRID_MIN,
    max_moves_per_target=99,
    debug=True
)

if plans.empty:
    print("\n✅ 제안 없음: 혼잡이 없거나(또는 보호구간에만 존재) 여유 도너가 부족합니다.")
else:
    print("\n🧭 재배치 제안(상위 30행):")
    print(plans.head(30).to_string(index=False))

    summary = plans.groupby("기준_날짜").agg(
        moved_total=("moved","sum"),
        unique_to_hours=("to_hour","nunique"),
        unique_from_hours=("from_hour","nunique")
    ).reset_index()
    print("\n📊 일자별 이동 요약:")
    print(summary.to_string(index=False))

    # 저장
    plans.to_csv(OUT_PLAN_PATH, index=False, encoding="utf-8-sig")
    print(f"\n💾 저장 완료: {OUT_PLAN_PATH}")

📥 Load schedule+allocation…
📥 Load hybrid τ…
[DEBUG] 타깃(혼잡) 시단 수: 15 / 도너(여유) 시단 수: 44
      보호구간(first+3, last-3) 적용 후 남은 시단만 대상으로 합니다.
[DEBUG] τ없음 → 2025-06-23 17h seq78 스킵
[DEBUG] τ없음 → 2025-06-23 18h seq78 스킵
[DEBUG] τ없음 → 2025-06-23 19h seq86 스킵
[DEBUG] τ없음 → 2025-06-24 17h seq79 스킵
[DEBUG] τ없음 → 2025-06-24 18h seq79 스킵
[DEBUG] τ없음 → 2025-06-25 17h seq78 스킵
[DEBUG] τ없음 → 2025-06-25 18h seq79 스킵
[DEBUG] τ없음 → 2025-06-26 18h seq78 스킵
[DEBUG] τ없음 → 2025-06-26 19h seq85 스킵
[DEBUG] τ없음 → 2025-06-26 17h seq78 스킵
[DEBUG] τ없음 → 2025-06-26 16h seq19 스킵

🧭 재배치 제안(상위 30행):
     기준_날짜  from_hour  to_hour  pivot_stop_seq     pivot_stop_name  needed_before  moved  needed_after           depart_time_snapped  tau_min   tau_source
2025-06-28          6       18              78  창덕궁.우리소리박물관(00078)              1      1             0 2025-06-28 15:50:00.000000400  128.596 observed_p60
2025-06-28         11       19              82 돈암사거리.성신여대입구(00082)              1      1             0 2025-06-28 16

In [8]:
import pandas as pd
import numpy as np
from math import ceil, floor

# ----------------------------------------------------
# 설정
# ----------------------------------------------------
BUS_DAY_PATH = "/Users/minwoo/Desktop/softeer/data_engineering_course_materials/missions/final_project/result/172_버스단위_스케줄+수요분배_2025-06-01_to_2025-06-30.csv"
OUT_PLAN_CSV = "/Users/minwoo/Desktop/softeer/data_engineering_course_materials/missions/final_project/result/원정류장기준_버스재배치_계획.csv"
OUT_DEPT_CSV = "/Users/minwoo/Desktop/softeer/data_engineering_course_materials/missions/final_project/result/원정류장기준_재배치후_출발스케줄.csv"

SEAT_CAP = 33.0
NORMAL_CAP = SEAT_CAP * 1.5   # 49.5
PROTECT_H = 3                 # 첫차+3h, 막차-3h 보호
MIN_KEEP_PER_HOUR = 1         # 도너 시간대 최소 유지 대수(0도 허용하려면 0)
# ----------------------------------------------------


# 요일 그룹(주중/주말) – 필요시 사용
def dow_group_from_date(s):
    dt = pd.to_datetime(s)
    wd = dt.weekday()  # 0=월 ... 6=일
    return "주말" if wd >= 5 else "주중"


# 보호구간 계산: 출발 정류장(정류장_순서==1)에서의 첫차/막차 시단
def compute_protected_windows(schedule_df, protect_h=3):
    src = schedule_df[schedule_df["정류장_순서"] == 1]
    guard = (
        src.groupby("기준_날짜")["도착_시간"]
           .agg(first_hour="min", last_hour="max")
           .reset_index()
    )
    guard["protect_start_hi"] = guard["first_hour"] + protect_h
    guard["protect_end_lo"]   = guard["last_hour"]  - protect_h
    return guard


# 시간대별 혼잡 타깃/도너 여유량 집계(원본 파일만으로 구성)
def build_pred_hourly(df):
    # 기준 컬럼 정리
    df = df.copy()
    if "도착_시간" not in df.columns and "도착시각" in df.columns:
        dt = pd.to_datetime(df["도착시각"])
        df["도착_시간"] = dt.dt.hour
    # bus_id 존재 가정
    if "onboard_after" not in df.columns:
        raise ValueError("입력 CSV에 onboard_after가 필요합니다(버스별 혼잡 판단용).")

    # 시간대-정류장 기준 집계(‘버스당_onboard_추정’은 버스별 on-board 평균으로 근사)
    grp_cols = ["기준_날짜", "도착_시간", "정류장_순서"]
    hourly = (
        df.groupby(grp_cols, as_index=False)
          .agg(
              통과버스수=("bus_id","nunique"),
              버스당_onboard_추정=("onboard_after","mean")
          )
    )
    # 피벗(각 (날짜,시간)에서 가장 혼잡한 정류장을 pivot으로)
    hourly["총_onboard_근사"] = hourly["통과버스수"] * hourly["버스당_onboard_추정"]
    # 타깃(혼잡) 포인트: 같은 (날짜,시간)에서 버스당_onboard_추정 최대인 정류장
    idx = hourly.groupby(["기준_날짜","도착_시간"])["버스당_onboard_추정"].idxmax()
    pivot = hourly.loc[idx].rename(columns={"도착_시간":"시간"})
    pivot["needed_before"] = (pivot["총_onboard_근사"] / NORMAL_CAP).apply(np.ceil) - pivot["통과버스수"]
    pivot["needed_before"] = pivot["needed_before"].clip(lower=0).astype(int)
    # 도너 여유: 같은 (날짜,시간)에서 제거해도 '여유' 유지 가능한 버스 수
    # spare = b - ceil(T/SEAT_CAP)
    hourly2 = hourly.rename(columns={"도착_시간":"시간"})
    hourly2["spare"] = hourly2["통과버스수"] - (hourly2["총_onboard_근사"] / SEAT_CAP).apply(np.ceil)
    hourly2["spare"] = hourly2["spare"].clip(lower=0).astype(int)

    return pivot[["기준_날짜","시간","정류장_순서","버스당_onboard_추정","통과버스수","총_onboard_근사","needed_before"]], \
           hourly2[["기준_날짜","시간","정류장_순서","버스당_onboard_추정","통과버스수","총_onboard_근사","spare"]]


# 도너 후보 버스 추출: (날짜,시간)의 출발정류장(순서==1)에서 출발한 bus_id 중, 여정 전체 최댓값이 33 이하
def donor_bus_candidates(day_df, donor_hour, df_full):
    # donor_hour에 출발한 버스들(정류장_순서==1 & 도착_시간==donor_hour)
    cond = (day_df["정류장_순서"]==1) & (day_df["도착_시간"]==donor_hour)
    bus_list = day_df.loc[cond, "bus_id"].unique().tolist()
    if not bus_list:
        return []

    # 각 bus_id의 여정 전체 max(onboard_after)
    trip_max = (df_full[df_full["bus_id"].isin(bus_list)]
                .groupby("bus_id")["onboard_after"].max())
    ok = trip_max[trip_max <= SEAT_CAP].sort_values()  # 낮은 max부터
    return ok.index.tolist()


# 균등 헤드웨이 출발 시각 생성(한 시간 안에서 n대를 60/n 간격으로)
def uniform_depart_times(day, hour, n):
    if n <= 0:
        return []
    base = pd.to_datetime(str(day)) + pd.Timedelta(hours=int(hour))
    step = 60.0 / n  # 분
    times = [base + pd.Timedelta(minutes=round(i*step, 3)) for i in range(n)]
    return times


# 메인: 원정류장 기준 버스 재배치
def reallocate_by_origin(bdf, protect_h=3, min_keep_per_hour=1, verbose=True):
    df = bdf.copy()
    if "도착_시간" not in df.columns and "도착시각" in df.columns:
        dt = pd.to_datetime(df["도착시각"])
        df["도착_시간"] = dt.dt.hour

    # 하루씩 처리 편의용
    days = sorted(df["기준_날짜"].unique())
    guard = compute_protected_windows(df, protect_h=protect_h)

    # 시간대 집계
    need_df, supply_df = build_pred_hourly(df)

    plans = []     # 이동 계획 (bus_id/시단 전후 포함)
    new_departs = []  # 재배치 후 출발 스케줄(균등 헤드웨이)

    for day in days:
        day_df = df[df["기준_날짜"]==day].copy()

        # 보호구간
        g = guard[guard["기준_날짜"]==day]
        if g.empty:
            continue
        lo = int(g.iloc[0]["protect_start_hi"])
        hi = int(g.iloc[0]["protect_end_lo"])

        # 타깃/도너 시간대 후보
        tgt_day = need_df[(need_df["기준_날짜"]==day) &
                          (need_df["needed_before"]>0) &
                          (need_df["시간"]>=lo) & (need_df["시간"]<=hi)].copy()

        don_day = supply_df[(supply_df["기준_날짜"]==day) &
                            (supply_df["spare"]>0) &
                            (supply_df["시간"]>=lo) & (supply_df["시간"]<=hi)].copy()

        if tgt_day.empty or don_day.empty:
            continue

        # 하루의 시간대별 출발 버스 수 테이블(정류장_순서==1)
        dep_cnt = (day_df[day_df["정류장_순서"]==1]
                   .groupby("도착_시간")["bus_id"].nunique()
                   .reindex(range(24), fill_value=0)
                   .astype(int))

        # 혼잡 큰 순(needed_before ↓), 도너는 여유 큰 순(spare ↓)
        tgt_day = tgt_day.sort_values(["needed_before","버스당_onboard_추정"], ascending=[False,False])
        don_day = don_day.sort_values(["spare","버스당_onboard_추정"], ascending=[False,True])

        for _, trow in tgt_day.iterrows():
            to_hour = int(trow["시간"])
            need = int(trow["needed_before"])
            if need <= 0:
                continue

            # 이 타깃을 채울 도너 시간대를 spare 큰 순으로 순회
            for __, drow in don_day.iterrows():
                if need <= 0:
                    break
                from_hour = int(drow["시간"])
                spare = int(drow["spare"])
                if spare <= 0:
                    continue

                # 도너에서 뺄 수 있는 수 = min(spare, dep_cnt[from_hour]-min_keep)
                can_take_from_hour = max(0, dep_cnt.get(from_hour, 0) - min_keep_per_hour)
                if can_take_from_hour <= 0:
                    continue
                take = min(spare, need, can_take_from_hour)

                # 도너 시간대의 실제 bus_id 후보(여정 max_onboard ≤ 33)
                candidates = donor_bus_candidates(day_df, from_hour, df)
                if not candidates:
                    continue

                # 필요한 수만큼 추출(후보가 부족하면 가능한 만큼만)
                picked = candidates[:take]
                # 상태 갱신
                need -= len(picked)
                drow["spare"] -= len(picked)
                dep_cnt[from_hour] -= len(picked)
                dep_cnt[to_hour]   += len(picked)

                # 계획에 기록(버스 식별자 포함)
                for bus in picked:
                    plans.append({
                        "기준_날짜": day,
                        "move_bus_id": bus,
                        "from_hour": from_hour,
                        "to_hour": to_hour
                    })

        # 이 날의 “재배치 후” 원정류장 출발 스케줄 생성(균등 헤드웨이)
        after_dep_table = []
        for h in range(24):
            n = int(dep_cnt.get(h, 0))
            times = uniform_depart_times(day, h, n)
            for k, ts in enumerate(times, start=1):
                after_dep_table.append({
                    "기준_날짜": day,
                    "도착_시간": h,
                    "slot_idx": k,
                    "균등_출발시각": ts
                })
        new_departs.append(pd.DataFrame(after_dep_table))

    plan_df = pd.DataFrame(plans)
    depart_df = pd.concat(new_departs, ignore_index=True) if new_departs else pd.DataFrame(
        columns=["기준_날짜","도착_시간","slot_idx","균등_출발시각"]
    )
    return plan_df, depart_df


# ----------------------------------------------------
# 실행
# ----------------------------------------------------
df = pd.read_csv(BUS_DAY_PATH)

# 기본 전처리: 타입/시간
if "도착_시간" not in df.columns and "도착시각" in df.columns:
    df["도착_시간"] = pd.to_datetime(df["도착시각"]).dt.hour

# 재배치 실행
plan_df, depart_df = reallocate_by_origin(
    bdf=df,
    protect_h=PROTECT_H,
    min_keep_per_hour=MIN_KEEP_PER_HOUR,
    verbose=True
)

# 저장 및 요약 출력
if plan_df.empty:
    print("✅ 재배치 제안이 없습니다(혼잡 없음, 또는 보호구간/여유 제약).")
else:
    # from→to 집계
    sum_day = plan_df.groupby("기준_날짜").size().reset_index(name="moved_buses")
    print("\n📊 일자별 이동 대수 요약:")
    print(sum_day.to_string(index=False))

    print(f"\n💾 계획 저장: {OUT_PLAN_CSV}")
    plan_df.to_csv(OUT_PLAN_CSV, index=False, encoding="utf-8-sig")

if not depart_df.empty:
    print(f"💾 재배치 후 원정류장 균등 출발 스케줄 저장: {OUT_DEPT_CSV}")
    depart_df.to_csv(OUT_DEPT_CSV, index=False, encoding="utf-8-sig")


📊 일자별 이동 대수 요약:
     기준_날짜  moved_buses
2025-06-23            3
2025-06-24            2
2025-06-25            2
2025-06-26            9
2025-06-28            2
2025-06-29            2

💾 계획 저장: /Users/minwoo/Desktop/softeer/data_engineering_course_materials/missions/final_project/result/원정류장기준_버스재배치_계획.csv
💾 재배치 후 원정류장 균등 출발 스케줄 저장: /Users/minwoo/Desktop/softeer/data_engineering_course_materials/missions/final_project/result/원정류장기준_재배치후_출발스케줄.csv
